In [1]:
# Import required libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import catboost
import xgboost
import lightgbm as lgb
import numpy as np
import os

# Read data files and pre-process
_DATA_DIR = './input'
    
logistics_data = 'contraceptive_logistics_data.csv'
product = 'product.csv'
site = 'service_delivery_site_data.csv'
submission_file = 'submission_format.csv'


class Dataset():
    """Monthly stock distribution data."""

    def __init__(self):
        """Load the data from disk."""

        self.logistics_data = pd.read_csv(os.path.join(_DATA_DIR, logistics_data))
        self.product = pd.read_csv(os.path.join(_DATA_DIR, product))
        self.site = pd.read_csv(os.path.join(_DATA_DIR, site))
        self.submission = pd.read_csv(os.path.join(_DATA_DIR,submission_file))

    def preprocess(self):
        """Preprocess the dataset."""

        # preprocessing operations go here.
        product_df = self._preprocess_product(self.product)
        site_df = self._preprocess_site(self.site)
        final_df,logistics_df,submission_ID,Zero_demand_df = self._preprocess_logistics(self.logistics_data,product_df,site_df,self.submission)

        return final_df,logistics_df,submission_ID,Zero_demand_df

    def _preprocess_product(self, product):
        product_df = product[['product_code', 'product_type']]

        return product_df

    def _preprocess_site(self, site):
        site_df = site[['site_code', 'site_type']]

        return site_df
    
    def _preprocess_logistics(self, logistics_data,product_df,site_df,submission):
        # Add day column
        logistics_data['day'] = 1

        # Create date column
        logistics_data['date'] = pd.to_datetime(logistics_data[['month', 'day', 'year']])
        logistics_data['date'] = logistics_data['date'].dt.strftime('%m/%d/%Y')
        logistics_data['date'] = pd.to_datetime(logistics_data['date'])

        logistics_data['batchID'] = logistics_data['product_code'] + "_" + logistics_data['site_code']

        # Rename column names
        logistics_data = logistics_data.rename(columns={'stock_distributed': 'value'})
        
        #Merge product and logistics data and bring the product type as feature
        logistics_data = pd.merge(logistics_data,product_df,how='left',left_on=['product_code'], right_on = ['product_code'])

        #Merge site type from service delivery site file as a feature
        logistics_data = pd.merge(logistics_data,site_df,how='left',left_on=['site_code'],right_on=['site_code'])
        
        # Drop columns
        logistics_data = logistics_data.drop(columns=['stock_received', 'stock_adjustment',
                                                      'stock_end', 'stock_stockout_days', 'stock_ordered'], axis=1)

        # Sort by date in ascending format
        logistics_data = logistics_data.sort_values(by='date', ascending=True)

        # Add missing time periods and fill missing values with 0
        logistics_data = (logistics_data.set_index('date')
                          .groupby('batchID')
                          .apply(lambda d: d.reindex(pd.date_range(min(logistics_data.date),
                                                                   max(logistics_data.date),
                                                                   freq='MS')))
                          .drop('batchID', axis=1)
                          .reset_index('batchID')
                          .fillna(0))
        
        # Set date as a column
        logistics_data['date'] = logistics_data.index

        a = logistics_data

        cols = ["region","district",'product_type','site_type',"site_code","product_code"]

        a[cols] = a[cols].replace({'0':np.nan, 0:np.nan})

        a = logistics_data.groupby('batchID')

        logistics_data['region'] = a['region'].transform(lambda s: s.loc[s.first_valid_index()])
        logistics_data['district'] = a['district'].transform(lambda s: s.loc[s.first_valid_index()])
        logistics_data['product_type'] = a['product_type'].transform(lambda s: s.loc[s.first_valid_index()])
        logistics_data['site_type'] = a['site_type'].transform(lambda s: s.loc[s.first_valid_index()])
        logistics_data['site_code'] = a['site_code'].transform(lambda s: s.loc[s.first_valid_index()])
        logistics_data['product_code'] = a['product_code'].transform(lambda s: s.loc[s.first_valid_index()])

        # Drop existing month, year and day columns with missing values and extract year from date
        logistics_data = logistics_data.drop(columns=['year', 'month', 'day'], axis=1)

        logistics_data['year'] = logistics_data.index.year
        logistics_data['month'] = logistics_data.index.month
        logistics_data['quarter'] = logistics_data.index.quarter

        # Convert month, year, and quarter as binary variables - convert to category
        logistics_data['month'] = "Month" + logistics_data['month'].astype(str)
        logistics_data['quarter'] = "Quarter" + logistics_data['quarter'].astype(str)
        logistics_data['year'] = "Year" + logistics_data['year'].astype(str)
        
        # Create unique product site combination
        submission['batchID'] = submission['product_code'] + "_" + submission['site_code']

        # List of unique batchID
        submission_ID = np.unique(submission['batchID'])

        #Product Site combinations to be predicted | Historical data available
        df_1 = logistics_data[logistics_data['batchID'].isin(submission_ID)]

        #Product site combination to be predicted and no historical data available
        NoHistorical_ID = np.unique(submission[~submission['batchID'].isin(np.unique(logistics_data['batchID']))]['batchID'])

        #Create dataframe for 45 months with 0 as demand for the product site combinations  
        Zero_demand_df = pd.DataFrame()

        for i in NoHistorical_ID:
            date = pd.date_range('1/1/2016', freq='MS', periods=45)
            a = pd.DataFrame(i, index=date, columns=['batchID'])
            a['date'] = a.index
            #a[['product_code','site_code']] = a.batchID.str.split("_",expand=True) 
            a['value'] = 0
            Zero_demand_df = Zero_demand_df.append(a)

        # Combine df_1 and df_2 to create historical data for all product site combinations requried to be forecasted
        Insample = pd.concat([df_1[['batchID','date','value']],Zero_demand_df])

        return Insample,df_1,submission_ID,Zero_demand_df


if __name__ == '__main__':
    # Make sure we can load the dataset
    dataset = Dataset()
    final_df,logistics_df,submission_ID,Zero_demand_df = dataset.preprocess()
    print('Successfully loaded the dataset.')

Successfully loaded the dataset.


In [2]:
# Split Train Test data: Forecast Horizon is 3, so last 3 data points from each group kept as test data
df_test = final_df.groupby('batchID').tail(3)

# Remove the Test data sets from Train data
comb = df_test['batchID']+df_test['date'].apply(str)
final_df['conc'] = final_df['batchID']+final_df['date'].apply(str)

df_train = final_df[~final_df['conc'].isin(comb)]

# Drop concatenate column
df_train = df_train.drop(columns = ['conc'],axis =1)

In [3]:
# Compute In sample Naive Forecasting Mean Absolute Error
from sklearn.metrics import mean_absolute_error

naive_forecasting_train = df_train

#Calculate in sample MAE - naive forecasting
naive_forecasting_train['prediction'] = naive_forecasting_train.groupby(['batchID'])['value'].shift(1)
naive_forecasting_train = naive_forecasting_train.fillna(0)
naive_forecasting_train['MAE'] = abs(naive_forecasting_train.prediction - naive_forecasting_train.value)

insample_naive = naive_forecasting_train.groupby('batchID')['MAE'].sum()/(naive_forecasting_train.groupby('batchID')['MAE'].count()-1)

#Overall average insample naive forecasting 
print("Average Insample error - naive forecasting :",insample_naive.mean())

Average Insample error - naive forecasting : 9.98532412130205


In [4]:
# In sample Naive MAE - Groupby for comparison
insample_naive = pd.DataFrame(naive_forecasting_train.groupby('batchID')['MAE'].mean())
insample_naive = insample_naive.reset_index()
insample_naive.columns = ['batchID','Insample-MAE']

In [5]:
# Find TS to be predicted and for which historical data is available
df = logistics_df

# Count number of zeroes per group
count = pd.DataFrame(df.groupby('batchID')['value'].agg(lambda x: x.eq(0).sum()))
count['batchID'] = count.index
count.columns = ['Zeroes','batchID']
count = count.reset_index(drop=True)

# Count length of time series per group
TSlength =  pd.DataFrame(df.groupby('batchID')['value'].count())
TSlength['batchID'] = TSlength.index
TSlength = TSlength.reset_index(drop=True)
TSlength.columns = ['count','batchID']

ZeroCount = count.merge(TSlength,on='batchID',how='inner')
ZeroCount['CountZero%'] = (ZeroCount['Zeroes']/ZeroCount['count'])*100

# Bin the % of zeroes into a new column
bins = [0, 20, 40, 60, 80, 100]
labels = [0, 20, 40, 60, 80]
categories = ['0','20','40','60','80']
ZeroCount['bins'] = pd.cut(ZeroCount['CountZero%'], bins = bins, labels = labels)
ZeroCount = ZeroCount.fillna(0)

# Filter out TS which have more than 80% of records as zeroes and add to Zero_df
Zero_df = pd.concat([df[df['batchID'].isin(ZeroCount[ZeroCount['bins'] == 80].batchID)],Zero_demand_df])
df = df[df['batchID'].isin(ZeroCount[ZeroCount['bins'] != 80].batchID)]

# Filter out TS with more than 40% and less than 80% of records as zeroes
Sixty_df = df[df['batchID'].isin(ZeroCount[ZeroCount['bins'] == 60].batchID)]
Forty_df = df[df['batchID'].isin(ZeroCount[ZeroCount['bins'] == 40].batchID)]

# Remove TS which have greater than 40% of records as zeroes - on remaining try machine learning algorithms
df = df[df['batchID'].isin(ZeroCount[ZeroCount['bins'] != 60].batchID)]
df = df[df['batchID'].isin(ZeroCount[ZeroCount['bins'] != 40].batchID)]

# Combine Forty and Sixty TS
Forty_Sixty_df = pd.concat([Forty_df,Sixty_df])

# Remove leading zeroes
NonZero_df = df[df.groupby('batchID')['value'].cumsum().gt(0)]
Forty_df = Forty_df[Forty_df.groupby('batchID')['value'].cumsum().gt(0)]
Sixty_df = Sixty_df[Sixty_df.groupby('batchID')['value'].cumsum().gt(0)]
Forty_Sixty_df = Forty_Sixty_df[Forty_Sixty_df.groupby('batchID')['value'].cumsum().gt(0)]

In [6]:
# Split Zero_df into Train and Test and fit naive forecasting.
Zero_df = Zero_df.sort_values(by = 'date',ascending=True)
Zero_df_Test = Zero_df.groupby('batchID').tail(3)

# Remove the Test data sets from Train data
combn = Zero_df_Test['batchID']+Zero_df_Test['date'].apply(str)
Zero_df['conc'] = Zero_df['batchID']+Zero_df['date'].apply(str)

Zero_df_Train = Zero_df[~Zero_df['conc'].isin(combn)]

# Drop concatenate column
Zero_df_Train = Zero_df_Train.drop(columns = ['conc'],axis = 1)

# Drop conc from df
Zero_df = Zero_df.drop(columns = ['conc'],axis=1)

In [7]:
#Naive Forecasting 
def naive_forecasting_test(Train,Test):
    '''Naive Forecasting 
    '''    
    # initialize empty dataset
    columns = ['batchID','Naive_MAE']
    datas = pd.DataFrame(columns=columns)
    
    # select unique batch_id
    batch_ids = Train['batchID'].unique()

    # groupby different batch_id
    Train = Train.groupby(['batchID'])
    Test = Test.groupby(['batchID'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        d_test = Test.get_group(batch_id)
        naive_train = np.asarray(d_train.value)
        naive_test = d_test[['batchID','value']]
        naive_test['naive'] = naive_train[len(naive_train)-1]
        naive_test['Naive_MAE'] = abs(naive_test.value - naive_test.naive)
        datas = datas.append(naive_test)
    
    return datas

In [8]:
# Fit Naive Forecasting on Zero_df 
# Drop date column as it is present in index
Zero_df_Naive_Train = Zero_df_Train.set_index('date')
Zero_df_Naive_Test = Zero_df_Test.set_index('date') 

# Fit Naive Forecasting Model on Zero df and Less than 5 data points and calculate out of sample MAE
Zero_df_Naive_Train = Zero_df_Naive_Train[['batchID','value']]
Zero_df_Naive_Test = Zero_df_Naive_Test[['batchID','value']]

# Fit and predict 
Zero_df_Naive_Forecast = naive_forecasting_test(Train = Zero_df_Naive_Train,Test = Zero_df_Naive_Test)

# Calculate in sample naive forecasting for each group
Zero_df_Naive_results = Zero_df_Naive_Forecast.groupby('batchID')['Naive_MAE'].mean()
Zero_df_Naive_results = Zero_df_Naive_results.reset_index()

print("Zero_df: Naive Forecasting - Average MAE",Zero_df_Naive_results['Naive_MAE'].mean())

Zero_df: Naive Forecasting - Average MAE 5.148936170212766


In [9]:
#Identify Zero_df TS with greater than MASE of 1 will be fit separately
res = pd.merge(Zero_df_Naive_results,insample_naive,on=['batchID'],how='inner')
res['MASE'] = res['Naive_MAE']/res['Insample-MAE']
res = res.replace([np.inf, -np.inf], 1)
res= res.fillna(1)
res.sort_values(by='MASE',ascending=False)
Zero_TS_NonZero_ID = res[res['MASE']>1].batchID

In [10]:
# Add TS with > 80% zero demand with MASE > 1 in Non-zero df
NonZero_df = pd.concat([logistics_df[logistics_df['batchID'].isin(Zero_TS_NonZero_ID)],NonZero_df])

In [11]:
# Fit Naive Forecasting on Zero_df  - on TS with MASE < 1
Zero_df_Naive_Train = Zero_df_Naive_Train[~Zero_df_Naive_Train['batchID'].isin(Zero_TS_NonZero_ID)]
Zero_df_Naive_Test = Zero_df_Naive_Test[~Zero_df_Naive_Test['batchID'].isin(Zero_TS_NonZero_ID)]

# Fit and predict 
Zero_df_Naive_Forecast = naive_forecasting_test(Train = Zero_df_Naive_Train,Test = Zero_df_Naive_Test)

# Calculate in sample naive forecasting for each group
Zero_df_Naive_results = Zero_df_Naive_Forecast.groupby('batchID')['Naive_MAE'].mean()
Zero_df_Naive_results = Zero_df_Naive_results.reset_index()

print("Zero_df Naive Forecasting: Average MAE",Zero_df_Naive_results['Naive_MAE'].mean())

# Merge insample naive to calculate MASE
res = pd.merge(Zero_df_Naive_results,insample_naive,on=['batchID'],how='inner')
res['MASE'] = res['Naive_MAE']/res['Insample-MAE']
res = res.replace([np.inf, -np.inf], 0)
res = res.fillna(1)
print("Zero_df MASE:",res['MASE'].mean())

Zero_df Naive Forecasting: Average MAE 0.9338731443994599
Zero_df MASE: 0.6293558237074743


In [12]:
#### Create Forecast dataframe from Oct to Dec'19 #######

# Non Zero_df
NonZero_Forecast_ID = np.unique(NonZero_df['batchID'])

#Create Forecast dataframe from Oct'19 to Dec'19   
NonZero_Forecast = pd.DataFrame()

for i in NonZero_Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['batchID'])
    a['date'] = a.index
    a['year'] = a.index.year
    a['month'] = a.index.month
    a['quarter'] = a.index.quarter 

    # Convert month, year, and quarter as binary variables - convert to category
    a['month'] = "Month"+a['month'].astype(str)
    a['quarter'] = "Quarter"+a['quarter'].astype(str)
    a['year'] = "Year"+a['year'].astype(str)
    a = a.reset_index(drop=True)
    a['value'] = 0
    a['average_monthly_consumption'] = 0
    a['stock_initial'] = 0
    NonZero_Forecast = NonZero_Forecast.append(a)

# Zero_df
Zero_df = Zero_df[~Zero_df['batchID'].isin(Zero_TS_NonZero_ID)] 
Zero_Forecast_ID = np.unique(Zero_df['batchID'])

#Create Forecast dataframe from Oct'19 to Dec'19   
Zero_Forecast = pd.DataFrame()

for i in Zero_Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['batchID'])
    Zero_Forecast = Zero_Forecast.append(a)

# Forty df 
Forty_Forecast_ID = np.unique(Forty_df['batchID'])

#Create Forecast dataframe from Oct'19 to Dec'19   
Forty_Forecast = pd.DataFrame()

for i in Forty_Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['batchID'])
    a['date'] = a.index
    a['year'] = a.index.year
    a['month'] = a.index.month
    a['quarter'] = a.index.quarter 

    # Convert month, year, and quarter as binary variables - convert to category
    a['month'] = "Month"+a['month'].astype(str)
    a['quarter'] = "Quarter"+a['quarter'].astype(str)
    a['year'] = "Year"+a['year'].astype(str)

    a = a.reset_index(drop=True)
    a['value'] = 0
    a['average_monthly_consumption'] = 0
    a['stock_initial'] = 0
    Forty_Forecast = Forty_Forecast.append(a)

# Sixty df
Sixty_Forecast_ID = np.unique(Sixty_df['batchID'])

#Create Forecast dataframe from Oct'19 to Dec'19   
Sixty_Forecast = pd.DataFrame()

for i in Sixty_Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['batchID'])
    a['date'] = a.index
    a['year'] = a.index.year
    a['month'] = a.index.month
    a['quarter'] = a.index.quarter 

    # Convert month, year, and quarter as binary variables - convert to category
    a['month'] = "Month"+a['month'].astype(str)
    a['quarter'] = "Quarter"+a['quarter'].astype(str)
    a['year'] = "Year"+a['year'].astype(str)

    a = a.reset_index(drop=True)
    a['value'] = 0
    a['average_monthly_consumption'] = 0
    a['stock_initial'] = 0
    Sixty_Forecast = Sixty_Forecast.append(a)

# Forty Sixty df 
Forty_Sixty_df_Forecast_ID = np.unique(Forty_Sixty_df['batchID'])

#Create Forecast dataframe from Oct'19 to Dec'19   
Forty_Sixty_df_Forecast = pd.DataFrame()

for i in Forty_Sixty_df_Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['batchID'])
    a['date'] = a.index
    a['year'] = a.index.year
    a['month'] = a.index.month
    a['quarter'] = a.index.quarter 

    # Convert month, year, and quarter as binary variables - convert to category
    a['month'] = "Month"+a['month'].astype(str)
    a['quarter'] = "Quarter"+a['quarter'].astype(str)
    a['year'] = "Year"+a['year'].astype(str)

    a = a.reset_index(drop=True)
    a['value'] = 0
    a['average_monthly_consumption'] = 0
    a['stock_initial'] = 0
    Forty_Sixty_df_Forecast = Forty_Sixty_df_Forecast.append(a)


In [13]:
#Naive Forecasting 
def naive_forecasting_Forecast(Train,Forecast):
    '''Naive Forecasting 
    '''    
    # initialize empty dataset
    columns = ['batchID','predicted_value']
    datas = pd.DataFrame(columns=columns)
    
    # select unique batch_id
    batch_ids = Train['batchID'].unique()

    # groupby different batch_id
    Train = Train.groupby(['batchID'])
    Forecast = Forecast.groupby(['batchID'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        d_Forecast = Forecast.get_group(batch_id)
        naive_train = np.asarray(d_train.value)
        naive_Forecast = d_Forecast[['batchID']]
        naive_Forecast['predicted_value'] = naive_train[len(naive_train)-1]
        datas = datas.append(naive_Forecast)
    
    return datas

In [14]:
# Naive Forecast for 247 TS - 80% of records have zeroes
Naive_Train = pd.concat([Zero_df_Naive_Train,Zero_df_Naive_Test])

# Forecast 
Naive_Forecast_Zero_df = naive_forecasting_Forecast(Train = Naive_Train,Forecast = Zero_Forecast)

# Add month and year as a column
Naive_Forecast_Zero_df['year'] = Naive_Forecast_Zero_df.index.year
Naive_Forecast_Zero_df['month'] = Naive_Forecast_Zero_df.index.month
Naive_Forecast_Zero_df = Naive_Forecast_Zero_df.reset_index(drop=True)

# Split batchID to product and site code and drop batchID
ID = Naive_Forecast_Zero_df["batchID"].str.split("_", n = 1, expand = True) 

Naive_Forecast_Zero_df['site_code']  = ID[1]
Naive_Forecast_Zero_df['product_code']  = ID[0]

Naive_Forecast_Zero_df = Naive_Forecast_Zero_df.drop(columns=['batchID'],axis=1)

In [15]:
# Add categorical columns from historic data to forecast columns
cat_cols = logistics_df.groupby(['batchID','site_type','region','district','site_code','product_code','product_type'])['value'].sum().reset_index()

# Merge cat cols to Forecast dataframe
NonZero_Forecast = pd.merge(NonZero_Forecast,cat_cols.drop(columns=['value'],axis=1),how='left',on=['batchID'])
Sixty_Forecast = pd.merge(Sixty_Forecast,cat_cols.drop(columns=['value'],axis=1),how='left',on=['batchID'])
Forty_Forecast = pd.merge(Forty_Forecast,cat_cols.drop(columns=['value'],axis=1),how='left',on=['batchID'])
Forty_Sixty_df_Forecast = pd.merge(Forty_Sixty_df_Forecast,cat_cols.drop(columns=['value'],axis=1),how='left',on=['batchID'])

In [16]:
##### Product Level Forecasting - Use as a causal to fit machine learning algorithm ########

#Product level aggregation
products = logistics_df.groupby(['product_code','date']).sum().reset_index()

# Split Train Test data: Forecast Horizon is 3, so last 3 data points from each group kept as test data
products_Test = products.groupby('product_code').tail(3)

# Remove the Test data sets from Train data
comb = products_Test['product_code']+products_Test['date'].apply(str)
products['conc'] = products['product_code']+products['date'].apply(str)

products_Train = products[~products['conc'].isin(comb)]

# Drop concatenate column
products_Train = products_Train.drop(columns = ['conc'],axis =1)

products_Train = products_Train[['product_code','date','value']]
products_Test = products_Test[['product_code','date','value']]

#Naive Forecasting 
def naive_forecasting(Train,Test):
    '''Naive Forecasting 
    '''    
    # initialize empty dataset
    columns = ['product_code','MAE']
    datas = pd.DataFrame(columns=columns)
    
    # select unique batch_id
    batch_ids = Train['product_code'].unique()

    # groupby different batch_id
    Train = Train.groupby(['product_code'])
    Test = Test.groupby(['product_code'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        d_test = Test.get_group(batch_id)
        naive_train = np.asarray(d_train.value)
        naive_test = d_test[['product_code','value']]
        naive_test['naive'] = naive_train[len(naive_train)-1]
        naive_test['MAE'] = abs(naive_test.value - naive_test.naive)
        #naive_test['MAE'] = mean_absolute_error(naive_test.value, naive_test.naive)
        #d_train = d_train[['batchID','Test','Pred','MAE']]
        datas = datas.append(naive_test)
    
    return datas

products_Train = products_Train.set_index('date')
products_Test = products_Test.set_index('date')

# Fit Naive Forecasting Model 
products_Train = products_Train[['product_code','value']]
products_Test = products_Test[['product_code','value']]

product_naive_test_causals = naive_forecasting(Train = products_Train,Test = products_Test)

product_naive_test_causals = product_naive_test_causals.reset_index()
product_naive_test_causals = product_naive_test_causals[['index','product_code','naive']]
product_naive_test_causals.columns = ['date','product_code','product_causal']

# List of site code to be forecasted
Forecast_ID = np.unique(products['product_code'])

#Create Forecast dataframe from Oct'19 to Dec'19   
Forecast = pd.DataFrame()

for i in Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['product_code'])
    a['date'] = a.index
    a = a.reset_index(drop=True)
    a['value'] = 0
    Forecast = Forecast.append(a)

#Naive Forecasting 
def naive_forecasting(Train,Forecast):
    '''Naive Forecasting 
    '''    
    # initialize empty dataset
    columns = ['date','product_code','product_causal']
    datas = pd.DataFrame(columns=columns)
    
    # select unique site_code
    batch_ids = Train['product_code'].unique()

    # groupby different site_code
    Train = Train.groupby(['product_code'])
    Forecast = Forecast.groupby(['product_code'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        test_results = Forecast.get_group(batch_id)
        naive_train = np.asarray(d_train.value)
        test_results['product_causal'] = naive_train[len(naive_train)-1]
        datas = datas.append(test_results)
    
    return datas

products_Train = products_Train.reset_index()
products_train = products_Train[['product_code','date','value']]
products_Forecast = Forecast[['product_code','date']]
product_naive_Forecast_causals = naive_forecasting(products_train,products_Forecast)

product_train_causals = products_Train
product_train_causals.columns = ['date','product_code','product_causal']
## output - product_naive_Forecast_causals, product_naive_test_causals,product_train_causals

In [17]:
##### Site Level Forecasting - Use as a causal to fit machine learning algorithm ########

#site level aggregation
site = logistics_df.groupby(['site_code','date']).sum().reset_index()

# Split Train Test data: Forecast Horizon is 3, so last 3 data points from each group kept as test data
site_Test = site.groupby('site_code').tail(3)

# Remove the Test data sets from Train data
comb = site_Test['site_code']+site_Test['date'].apply(str)
site['conc'] = site['site_code']+site['date'].apply(str)

site_Train = site[~site['conc'].isin(comb)]

# Drop concatenate column
site_Train = site_Train.drop(columns = ['conc'],axis =1)

site_Train = site_Train[['site_code','date','value']]
site_Test = site_Test[['site_code','date','value']]

#Naive Forecasting 
def naive_forecasting(Train,Test):
    '''Naive Forecasting 
    '''    
    # initialize empty dataset
    columns = ['site_code','MAE']
    datas = pd.DataFrame(columns=columns)
    
    # select unique batch_id
    batch_ids = Train['site_code'].unique()

    # groupby different batch_id
    Train = Train.groupby(['site_code'])
    Test = Test.groupby(['site_code'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        d_test = Test.get_group(batch_id)
        naive_train = np.asarray(d_train.value)
        naive_test = d_test[['site_code','value']]
        naive_test['naive'] = naive_train[len(naive_train)-1]
        naive_test['MAE'] = abs(naive_test.value - naive_test.naive)
        #naive_test['MAE'] = mean_absolute_error(naive_test.value, naive_test.naive)
        #d_train = d_train[['batchID','Test','Pred','MAE']]
        datas = datas.append(naive_test)
    
    return datas

site_Train = site_Train.set_index('date')
site_Test = site_Test.set_index('date')

# Fit Naive Forecasting Model 
site_Train = site_Train[['site_code','value']]
site_Test = site_Test[['site_code','value']]

site_naive_test_causals = naive_forecasting(Train = site_Train,Test = site_Test)

site_naive_test_causals = site_naive_test_causals.reset_index()
site_naive_test_causals = site_naive_test_causals[['index','site_code','naive']]
site_naive_test_causals.columns = ['date','site_code','site_causal']

# List of site code to be forecasted
Forecast_ID = np.unique(site['site_code'])

#Create Forecast dataframe from Oct'19 to Dec'19   
Forecast = pd.DataFrame()

for i in Forecast_ID:
    # Date range from Oct'19 to Dec'19
    date = pd.date_range('10/1/2019', freq='MS', periods=3)
    a = pd.DataFrame(i, index=date, columns=['site_code'])
    a['date'] = a.index
    a = a.reset_index(drop=True)
    a['value'] = 0
    Forecast = Forecast.append(a)

#Naive Forecasting 
def naive_forecasting(Train,Forecast):
    '''Naive Forecasting 
    '''    
    # initialize empty dataset
    columns = ['date','site_code','site_causal']
    datas = pd.DataFrame(columns=columns)
    
    # select unique site_code
    batch_ids = Train['site_code'].unique()

    # groupby different site_code
    Train = Train.groupby(['site_code'])
    Forecast = Forecast.groupby(['site_code'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        test_results = Forecast.get_group(batch_id)
        naive_train = np.asarray(d_train.value)
        test_results['site_causal'] = naive_train[len(naive_train)-1]
        datas = datas.append(test_results)
    
    return datas

site_Train = site_Train.reset_index()
site_train = site_Train[['site_code','date','value']]
site_Forecast = Forecast[['site_code','date']]
site_naive_Forecast_causals = naive_forecasting(site_train,site_Forecast)

site_train_causals = site_Train
site_train_causals.columns = ['date','site_code','site_causal']
## output - site_naive_Forecast_causals, site_naive_test_causals,site_train_causals

In [18]:
# Concatenate train and test causals - product and site
product_causals = pd.concat([product_train_causals,product_naive_test_causals])
site_causals = pd.concat([site_train_causals,site_naive_test_causals])

# Merge product and site causals with df
NonZero_df = pd.merge(NonZero_df,product_causals,how='left',on=['product_code','date'])
NonZero_df = pd.merge(NonZero_df,site_causals,how='left',on=['site_code','date'])

Forty_Sixty_df = pd.merge(Forty_Sixty_df,product_causals,how='left',on=['product_code','date'])
Forty_Sixty_df = pd.merge(Forty_Sixty_df,site_causals,how='left',on=['site_code','date'])

Forty_df = pd.merge(Forty_df,product_causals,how='left',on=['product_code','date'])
Sixty_df = pd.merge(Sixty_df,product_causals,how='left',on=['product_code','date'])

Forty_df = pd.merge(Forty_df,site_causals,how='left',on=['site_code','date'])
Sixty_df = pd.merge(Sixty_df,site_causals,how='left',on=['site_code','date'])

In [19]:
# Add product and site forecast causals to forecast data
NonZero_Forecast = pd.merge(NonZero_Forecast,product_naive_Forecast_causals,how='left',on=['product_code','date'])
NonZero_Forecast = pd.merge(NonZero_Forecast,site_naive_Forecast_causals,how='left',on=['site_code','date'])

Forty_Sixty_df_Forecast = pd.merge(Forty_Sixty_df_Forecast,product_naive_Forecast_causals,how='left',on=['product_code','date'])
Forty_Sixty_df_Forecast = pd.merge(Forty_Sixty_df_Forecast,site_naive_Forecast_causals,how='left',on=['site_code','date'])

Forty_Forecast = pd.merge(Forty_Forecast,product_naive_Forecast_causals,how='left',on=['product_code','date'])
Sixty_Forecast = pd.merge(Sixty_Forecast,product_naive_Forecast_causals,how='left',on=['product_code','date'])

Forty_Forecast = pd.merge(Forty_Forecast,site_naive_Forecast_causals,how='left',on=['site_code','date'])
Sixty_Forecast = pd.merge(Sixty_Forecast,site_naive_Forecast_causals,how='left',on=['site_code','date'])

In [20]:
# Sort values by date
NonZero_df = NonZero_df.sort_values(by = 'date',ascending=True)
Forty_Sixty_df = Forty_Sixty_df.sort_values(by = 'date',ascending=True)
Forty_df = Forty_df.sort_values(by = 'date',ascending=True)
Sixty_df = Sixty_df.sort_values(by = 'date',ascending=True)

# Split data into train and test
NonZero_df_Test = NonZero_df.groupby('batchID').tail(3)
Forty_Sixty_df_Test = Forty_Sixty_df.groupby('batchID').tail(3)
Forty_df_Test = Forty_df.groupby('batchID').tail(3)
Sixty_df_Test = Sixty_df.groupby('batchID').tail(3)

##################Forty_df#######################
# Remove the Test data sets from Train data
combn = Forty_df_Test['batchID']+Forty_df_Test['date'].apply(str)
Forty_df['conc'] = Forty_df['batchID']+Forty_df['date'].apply(str)

Forty_df_Train = Forty_df[~Forty_df['conc'].isin(combn)]

# Drop concatenate column
Forty_df_Train = Forty_df_Train.drop(columns = ['conc'],axis = 1)


################# Forty Sixty_df#######################
# Remove the Test data sets from Train data
combn = Forty_Sixty_df_Test['batchID']+Forty_Sixty_df_Test['date'].apply(str)
Forty_Sixty_df['conc'] = Forty_Sixty_df['batchID']+Forty_Sixty_df['date'].apply(str)

Forty_Sixty_df_Train = Forty_Sixty_df[~Forty_Sixty_df['conc'].isin(combn)]

# Drop concatenate column
Forty_Sixty_df_Train = Forty_Sixty_df_Train.drop(columns = ['conc'],axis = 1)

################# Sixty_df#######################
# Remove the Test data sets from Train data
combn = Sixty_df_Test['batchID']+Sixty_df_Test['date'].apply(str)
Sixty_df['conc'] = Sixty_df['batchID']+Sixty_df['date'].apply(str)

Sixty_df_Train = Sixty_df[~Sixty_df['conc'].isin(combn)]

# Drop concatenate column
Sixty_df_Train = Sixty_df_Train.drop(columns = ['conc'],axis = 1)

#################NonZero_df###########################
# Remove the Test data sets from Train data
combn = NonZero_df_Test['batchID']+NonZero_df_Test['date'].apply(str)
NonZero_df['conc'] = NonZero_df['batchID']+NonZero_df['date'].apply(str)

NonZero_df_Train = NonZero_df[~NonZero_df['conc'].isin(combn)]

# Drop concatenate column
NonZero_df_Train = NonZero_df_Train.drop(columns = ['conc'],axis = 1)

NonZero_df = NonZero_df.drop(columns = ['conc'],axis=1)

In [21]:
# Time Series Decomposition
def decompose(df):
    '''Replace out of bound dates batch_id wise 
    '''
    # initialize empty dataset
    columns = ['batchID','value']
    #datas = [] 
    datas = pd.DataFrame(columns=columns)
    
    # select unique batch_id
    batch_ids = np.unique(df['batchID'])

    # groupby different batch_id
    df = df.groupby(['batchID'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        df_partial = df.get_group(batch_id)
        try:
            result = seasonal_decompose(df_partial.value, model='additive',freq = 12)
        except:
            continue
        df_partial['trend'] = result.trend
        df_partial['seasonal'] = result.seasonal
        df_partial['resid'] = result.resid
        df_partial['observed'] = result.observed
        datas = datas.append(df_partial)
    
    return datas

In [22]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Non Zero_df, Forty_df, Sixty_df
NonZero_decomposition = decompose(NonZero_df_Train)
Forty_Sixty_df_decomposition = decompose(Forty_Sixty_df_Train)
Forty_decomposition = decompose(Forty_df_Train)
Sixty_decomposition = decompose(Sixty_df_Train)

# Merge Seasonality as causal 
NonZero_df_Train = pd.merge(NonZero_df_Train,NonZero_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)
Forty_Sixty_df_Train = pd.merge(Forty_Sixty_df_Train,Forty_Sixty_df_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)
Forty_df_Train = pd.merge(Forty_df_Train,Forty_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)
Sixty_df_Train = pd.merge(Sixty_df_Train,Sixty_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)

# Seasonal on entire data
NonZero_full_decomposition = decompose(NonZero_df)
Forty_Sixty_df_full_decomposition = decompose(Forty_Sixty_df)
Forty_full_decomposition = decompose(Forty_df)
Sixty_full_decomposition = decompose(Sixty_df)

# Merge Seasonality as causal 
NonZero_df = pd.merge(NonZero_df,NonZero_full_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)
Forty_Sixty_df = pd.merge(Forty_Sixty_df,Forty_Sixty_df_full_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)
Forty_df = pd.merge(Forty_df,Forty_full_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)
Sixty_df = pd.merge(Sixty_df,Sixty_full_decomposition[['batchID','date','seasonal']],how='left',on=['batchID','date']).fillna(0)


In [23]:
# Add seasonal causal to test data and forecast
NonZero_seasonal_test = NonZero_df_Train.groupby(['batchID','month'])['seasonal'].mean().reset_index()
Forty_Sixty_df_seasonal_test = Forty_Sixty_df_Train.groupby(['batchID','month'])['seasonal'].mean().reset_index()
Forty_seasonal_test = Forty_df_Train.groupby(['batchID','month'])['seasonal'].mean().reset_index()
Sixty_seasonal_test = Sixty_df_Train.groupby(['batchID','month'])['seasonal'].mean().reset_index()

# Add seasonal prediction to test data based on batchID and month
NonZero_df_Test = pd.merge(NonZero_df_Test,NonZero_seasonal_test,how='left',on=['batchID','month'])
Forty_Sixty_df_Test = pd.merge(Forty_Sixty_df_Test,Forty_Sixty_df_seasonal_test,how='left',on=['batchID','month'])
Forty_df_Test = pd.merge(Forty_df_Test,Forty_seasonal_test,how='left',on=['batchID','month'])
Sixty_df_Test = pd.merge(Sixty_df_Test,Sixty_seasonal_test,how='left',on=['batchID','month'])

#Add Seasonal values to Forecast data
NonZero_seasonal = NonZero_df.groupby(['batchID','month'])['seasonal'].mean().reset_index()
Forty_Sixty_df_seasonal = Forty_Sixty_df.groupby(['batchID','month'])['seasonal'].mean().reset_index()
Forty_seasonal = Forty_df.groupby(['batchID','month'])['seasonal'].mean().reset_index()
Sixty_seasonal = Sixty_df.groupby(['batchID','month'])['seasonal'].mean().reset_index()

# Add seasonal prediction to test data based on batchID and month
NonZero_Forecast = pd.merge(NonZero_Forecast,NonZero_seasonal,how='left',on=['batchID','month'])
Forty_Sixty_df_Forecast = pd.merge(Forty_Sixty_df_Forecast,Forty_Sixty_df_seasonal,how='left',on=['batchID','month'])
Forty_Forecast = pd.merge(Forty_Forecast,Forty_seasonal,how='left',on=['batchID','month'])
Sixty_Forecast = pd.merge(Sixty_Forecast,Sixty_seasonal,how='left',on=['batchID','month'])


In [24]:
# Fillna values with 0
Sixty_Forecast = Sixty_Forecast.fillna(0)
Forty_Forecast = Forty_Forecast.fillna(0)
Forty_Sixty_df_Forecast = Forty_Sixty_df_Forecast.fillna(0)
NonZero_Forecast = NonZero_Forecast.fillna(0)

# Fillna values with 0
Sixty_Forecast = Sixty_Forecast.fillna(0)
Forty_Forecast = Forty_Forecast.fillna(0)
Forty_Sixty_df_Forecast = Forty_Sixty_df_Forecast.fillna(0)
NonZero_Forecast = NonZero_Forecast.fillna(0)


In [25]:
# Drop date column as it is present in index
Forty_Sixty_df_Naive_Train = Forty_Sixty_df_Train.set_index('date')
Forty_Sixty_df_Naive_Test = Forty_Sixty_df_Test.set_index('date') 

Forty_Sixty_df_Naive_Train = Forty_Sixty_df_Naive_Train[['batchID','value']]
Forty_Sixty_df_Naive_Test = Forty_Sixty_df_Naive_Test[['batchID','value']]

# Fit and predict 
Forty_Sixty_df_Naive_Forecast = naive_forecasting_test(Train = Forty_Sixty_df_Naive_Train,Test = Forty_Sixty_df_Naive_Test)

# Calculate in sample naive forecasting for each group
Forty_Sixty_df_Naive_results = Forty_Sixty_df_Naive_Forecast.groupby('batchID')['Naive_MAE'].mean()
Forty_Sixty_df_Naive_results = Forty_Sixty_df_Naive_results.reset_index()

print("Average MAE Forty_Sixty_df -Naive Forecasting:",Forty_Sixty_df_Naive_results['Naive_MAE'].mean())

Average MAE Forty_Sixty_df -Naive Forecasting: 11.637457044673539


In [26]:
# Drop date column as it is present in index
Sixty_df_Naive_Train = Sixty_df_Train.set_index('date')
Sixty_df_Naive_Test = Sixty_df_Test.set_index('date') 

Sixty_df_Naive_Train = Sixty_df_Naive_Train[['batchID','value']]
Sixty_df_Naive_Test = Sixty_df_Naive_Test[['batchID','value']]

# Fit and predict 
Sixty_df_Naive_Forecast = naive_forecasting_test(Train = Sixty_df_Naive_Train,Test = Sixty_df_Naive_Test)

# Calculate in sample naive forecasting for each group
Sixty_df_Naive_results = Sixty_df_Naive_Forecast.groupby('batchID')['Naive_MAE'].mean()
Sixty_df_Naive_results = Sixty_df_Naive_results.reset_index()

print("Average MAE -Sixty_df -Naive Forecasting:",Sixty_df_Naive_results['Naive_MAE'].mean())

Average MAE -Sixty_df -Naive Forecasting: 7.6787878787878805


In [27]:
# Drop date column as it is present in index
Forty_df_Naive_Train = Forty_df_Train.set_index('date')
Forty_df_Naive_Test = Forty_df_Test.set_index('date') 

# Fit Naive Forecasting Model on Zero df and Less than 5 data points and calculate out of sample MAE
Forty_df_Naive_Train = Forty_df_Naive_Train[['batchID','value']]
Forty_df_Naive_Test = Forty_df_Naive_Test[['batchID','value']]

# Fit and predict 
Forty_df_Naive_Forecast = naive_forecasting_test(Train = Forty_df_Naive_Train,Test = Forty_df_Naive_Test)

# Calculate in sample naive forecasting for each group
Forty_df_Naive_results = Forty_df_Naive_Forecast.groupby('batchID')['Naive_MAE'].mean()
Forty_df_Naive_results = Forty_df_Naive_results.reset_index()

print("Average MAE - Forty_df -Naive Forecasting:",Forty_df_Naive_results['Naive_MAE'].mean())

Average MAE - Forty_df -Naive Forecasting: 14.566517189835574


In [28]:
# Drop date column as it is present in index
NonZero_df_Naive_Train = NonZero_df_Train.set_index('date')
NonZero_df_Naive_Test = NonZero_df_Test.set_index('date') 

# Fit Naive Forecasting Model on Zero df and Less than 5 data points and calculate out of sample MAE
NonZero_df_Naive_Train = NonZero_df_Naive_Train[['batchID','value']]
NonZero_df_Naive_Test = NonZero_df_Naive_Test[['batchID','value']]

# Fit and predict 
NonZero_df_Naive_Forecast = naive_forecasting_test(Train = NonZero_df_Naive_Train,Test = NonZero_df_Naive_Test)

# Calculate in sample naive forecasting for each group
NonZero_df_Naive_results = NonZero_df_Naive_Forecast.groupby('batchID')['Naive_MAE'].mean()
NonZero_df_Naive_results = NonZero_df_Naive_results.reset_index()

print("Average MAE - Zero_df Naive Forecasting:",NonZero_df_Naive_results['Naive_MAE'].mean())

Average MAE - Zero_df Naive Forecasting: 18.46682653876898


In [29]:
#Intermittent Forecasting 
def Croston_Intermittent_forecasting_test(Train,Test):
    '''Intermitten Forecasting - Using Croston's method 
    '''    
    # initialize empty dataset
    columns = ['batchID','Croston_MAE']
    datas = pd.DataFrame(columns=columns)
    
    # select unique batch_id
    batch_ids = Train['batchID'].unique()

    # groupby different batch_id
    Train = Train.groupby(['batchID'])
    Test = Test.groupby(['batchID'])
    
    #loop through the batch_ids
    for batch_id in batch_ids:
        d_train = Train.get_group(batch_id)
        d_test = Test.get_group(batch_id)
        int_train = np.asarray(d_train.value)
        pred = croston.fit_croston(int_train,3,'original')['croston_forecast']
        int_test = d_test[['batchID','value']]
        int_test['forecast'] = pred
        int_test['Croston_MAE'] = abs(int_test.value - int_test.forecast)
        datas = datas.append(int_test)
    
    return datas

In [30]:
from croston import croston
# Non Zero_df Croston
NonZero_Croston = Croston_Intermittent_forecasting_test(NonZero_df_Train,NonZero_df_Test)
#NonZero_Croston = NonZero_Croston.fillna(0)
print("Croston's Intermittent Forecasting:",NonZero_Croston['Croston_MAE'].mean())
NonZero_Croston_MAE = NonZero_Croston.groupby('batchID')['Croston_MAE'].mean()

# Sixty df Croston
Forty_Sixty_df_Croston = Croston_Intermittent_forecasting_test(Forty_Sixty_df_Train,Forty_Sixty_df_Test)
#Forty_Sixty_df_Croston = Forty_Sixty_df_Croston.fillna(0)
print("Croston's Intermittent Forecasting:",Forty_Sixty_df_Croston['Croston_MAE'].mean())
Forty_Sixty_df_Croston_MAE = Forty_Sixty_df_Croston.groupby('batchID')['Croston_MAE'].mean()

# Sixty df Croston
Sixty_Croston = Croston_Intermittent_forecasting_test(Sixty_df_Train,Sixty_df_Test)
#Sixty_Croston = Sixty_Croston.fillna(0)
print("Croston's Intermittent Forecasting:",Sixty_Croston['Croston_MAE'].mean())
Sixty_Croston_MAE = Sixty_Croston.groupby('batchID')['Croston_MAE'].mean()

# Forty df Croston
Forty_Croston = Croston_Intermittent_forecasting_test(Forty_df_Train,Forty_df_Test)
#Forty_Croston = Forty_Croston.fillna(0)
print("Croston's Intermittent Forecasting:",Forty_Croston['Croston_MAE'].mean())
Forty_Croston_MAE = Forty_Croston.groupby('batchID')['Croston_MAE'].mean()

Croston's Intermittent Forecasting: 17.22281179843117
Croston's Intermittent Forecasting: 12.098562325086325
Croston's Intermittent Forecasting: 9.420381537240951
Croston's Intermittent Forecasting: 14.080175912505519


In [31]:
#################### Non Zero df - Calculate lags ##################

NonZero_df_Train['tag'] = 'Train'
NonZero_df_Test['tag'] = 'Test'
NonZero_Forecast['tag'] = 'Forecast'

#Combine train and test to calculate demand lags and SMA lags
NonZero_df_lag = pd.concat([NonZero_df_Train,NonZero_df_Test,NonZero_Forecast])

#Add demand lags 
for i in range(3, 8):
    NonZero_df_lag["dm_lag_{}".format(i)] = NonZero_df_lag.groupby(['batchID'])['value'].shift(i)

# Add average monthly consumption lags
for i in range(3, 5):
    NonZero_df_lag["avg_conmptn_lag_{}".format(i)] = NonZero_df_lag.groupby(['batchID'])['average_monthly_consumption'].shift(i)
        
# Add opening stock lags
for i in range(3, 5):
    NonZero_df_lag["Stock_Initial_lag{}".format(i)] = NonZero_df_lag.groupby(['batchID'])['stock_initial'].shift(i)

# Calculate SMA lags
NonZero_df_lag['SMA_6'] = (NonZero_df_lag['dm_lag_3']+NonZero_df_lag['dm_lag_4']+NonZero_df_lag['dm_lag_5'])/3
NonZero_df_lag['SMA_7'] = (NonZero_df_lag['dm_lag_4']+NonZero_df_lag['dm_lag_5']+NonZero_df_lag['dm_lag_6'])/3
NonZero_df_lag['SMA_8'] = (NonZero_df_lag['dm_lag_5']+NonZero_df_lag['dm_lag_6']+NonZero_df_lag['dm_lag_7'])/3

#drop demand lags, average monthly consumption, initial stock columns
NonZero_df_lag = NonZero_df_lag.drop(columns = ['average_monthly_consumption','stock_initial','product_type','site_type','region'],axis=1)

# Split NonZero_df_lag into Train, test and forecast
NonZero_df_Train = NonZero_df_lag[NonZero_df_lag['tag']=='Train']
NonZero_df_Test = NonZero_df_lag[NonZero_df_lag['tag']=='Test']
NonZero_Forecast = NonZero_df_lag[NonZero_df_lag['tag']=='Forecast']

#Remove tag column
NonZero_df_Train = NonZero_df_Train.drop(columns = ['tag'],axis = 1)
NonZero_df_Test = NonZero_df_Test.drop(columns = ['tag'],axis = 1)
NonZero_Forecast = NonZero_Forecast.drop(columns = ['tag'],axis = 1)

NonZero_df_Train = NonZero_df_Train.fillna(0)
NonZero_df_Test = NonZero_df_Test.fillna(0)
NonZero_Forecast = NonZero_Forecast.fillna(0)

In [32]:
#################### Forty Sixty df - Calculate lags ##################

Forty_Sixty_df_Train['tag'] = 'Train'
Forty_Sixty_df_Test['tag'] = 'Test'
Forty_Sixty_df_Forecast['tag'] = 'Forecast'

#Combine train and test to calculate demand lags and SMA lags
Forty_Sixty_df_lag = pd.concat([Forty_Sixty_df_Train,Forty_Sixty_df_Test,Forty_Sixty_df_Forecast])

#Add demand lags 
for i in range(3, 8):
    Forty_Sixty_df_lag["dm_lag_{}".format(i)] = Forty_Sixty_df_lag.groupby(['batchID'])['value'].shift(i)

# Add average monthly consumption lags
for i in range(3, 5):
    Forty_Sixty_df_lag["avg_conmptn_lag_{}".format(i)] = Forty_Sixty_df_lag.groupby(['batchID'])['average_monthly_consumption'].shift(i)
        
# Add opening stock lags
for i in range(3, 5):
    Forty_Sixty_df_lag["Stock_Initial_lag{}".format(i)] = Forty_Sixty_df_lag.groupby(['batchID'])['stock_initial'].shift(i)

# Calculate SMA lags
Forty_Sixty_df_lag['SMA_6'] = (Forty_Sixty_df_lag['dm_lag_3']+Forty_Sixty_df_lag['dm_lag_4']+Forty_Sixty_df_lag['dm_lag_5'])/3
Forty_Sixty_df_lag['SMA_7'] = (Forty_Sixty_df_lag['dm_lag_4']+Forty_Sixty_df_lag['dm_lag_5']+Forty_Sixty_df_lag['dm_lag_6'])/3
Forty_Sixty_df_lag['SMA_8'] = (Forty_Sixty_df_lag['dm_lag_5']+Forty_Sixty_df_lag['dm_lag_6']+Forty_Sixty_df_lag['dm_lag_7'])/3

#drop demand lags, average monthly consumption, initial stock columns
Forty_Sixty_df_lag = Forty_Sixty_df_lag.drop(columns = ['average_monthly_consumption','stock_initial','product_type','site_type','region'],axis=1)

# Split NonZero_df_lag into Train, test and forecast
Forty_Sixty_df_Train = Forty_Sixty_df_lag[Forty_Sixty_df_lag['tag']=='Train']
Forty_Sixty_df_Test = Forty_Sixty_df_lag[Forty_Sixty_df_lag['tag']=='Test']
Forty_Sixty_df_Forecast = Forty_Sixty_df_lag[Forty_Sixty_df_lag['tag']=='Forecast']

#Remove tag column
Forty_Sixty_df_Train = Forty_Sixty_df_Train.drop(columns = ['tag'],axis = 1)
Forty_Sixty_df_Test = Forty_Sixty_df_Test.drop(columns = ['tag'],axis = 1)
Forty_Sixty_df_Forecast = Forty_Sixty_df_Forecast.drop(columns = ['tag'],axis = 1)

Forty_Sixty_df_Train = Forty_Sixty_df_Train.fillna(0)
Forty_Sixty_df_Test = Forty_Sixty_df_Test.fillna(0)
Forty_Sixty_df_Forecast = Forty_Sixty_df_Forecast.fillna(0)

In [33]:
#################### Sixty df - Calculate lags ##################

Sixty_df_Train['tag'] = 'Train'
Sixty_df_Test['tag'] = 'Test'
Sixty_Forecast['tag'] = 'Forecast'

#Combine train and test to calculate demand lags and SMA lags
Sixty_df_lag = pd.concat([Sixty_df_Train,Sixty_df_Test,Sixty_Forecast])

#Add demand lags 
for i in range(3, 8):
    Sixty_df_lag["dm_lag_{}".format(i)] = Sixty_df_lag.groupby(['batchID'])['value'].shift(i)

# Add average monthly consumption lags
for i in range(3, 5):
    Sixty_df_lag["avg_conmptn_lag_{}".format(i)] = Sixty_df_lag.groupby(['batchID'])['average_monthly_consumption'].shift(i)
        
# Add opening stock lags
for i in range(3, 5):
    Sixty_df_lag["Stock_Initial_lag{}".format(i)] = Sixty_df_lag.groupby(['batchID'])['stock_initial'].shift(i)

# Calculate SMA lags
Sixty_df_lag['SMA_6'] = (Sixty_df_lag['dm_lag_3']+Sixty_df_lag['dm_lag_4']+Sixty_df_lag['dm_lag_5'])/3
Sixty_df_lag['SMA_7'] = (Sixty_df_lag['dm_lag_4']+Sixty_df_lag['dm_lag_5']+Sixty_df_lag['dm_lag_6'])/3
Sixty_df_lag['SMA_8'] = (Sixty_df_lag['dm_lag_5']+Sixty_df_lag['dm_lag_6']+Sixty_df_lag['dm_lag_7'])/3

#drop demand lags, average monthly consumption, initial stock columns
Sixty_df_lag = Sixty_df_lag.drop(columns = ['average_monthly_consumption','stock_initial','product_type','site_type','region'],axis=1)

# Split NonZero_df_lag into Train, test and forecast
Sixty_df_Train = Sixty_df_lag[Sixty_df_lag['tag']=='Train']
Sixty_df_Test = Sixty_df_lag[Sixty_df_lag['tag']=='Test']
Sixty_Forecast = Sixty_df_lag[Sixty_df_lag['tag']=='Forecast']

#Remove tag column
Sixty_df_Train = Sixty_df_Train.drop(columns = ['tag'],axis = 1)
Sixty_df_Test = Sixty_df_Test.drop(columns = ['tag'],axis = 1)
Sixty_Forecast = Sixty_Forecast.drop(columns = ['tag'],axis = 1)

Sixty_df_Train = Sixty_df_Train.fillna(0)
Sixty_df_Test = Sixty_df_Test.fillna(0)
Sixty_Forecast = Sixty_Forecast.fillna(0)

In [34]:
#################### Forty df - Calculate lags ##################

Forty_df_Train['tag'] = 'Train'
Forty_df_Test['tag'] = 'Test'
Forty_Forecast['tag'] = 'Forecast'

#Combine train and test to calculate demand lags and SMA lags
Forty_df_lag = pd.concat([Forty_df_Train,Forty_df_Test,Forty_Forecast])

#Add demand lags 
for i in range(3, 8):
    Forty_df_lag["dm_lag_{}".format(i)] = Forty_df_lag.groupby(['batchID'])['value'].shift(i)

# Add average monthly consumption lags
for i in range(3, 5):
    Forty_df_lag["avg_conmptn_lag_{}".format(i)] = Forty_df_lag.groupby(['batchID'])['average_monthly_consumption'].shift(i)
        
# Add opening stock lags
for i in range(3, 5):
    Forty_df_lag["Stock_Initial_lag{}".format(i)] = Forty_df_lag.groupby(['batchID'])['stock_initial'].shift(i)

# Calculate SMA lags
Forty_df_lag['SMA_6'] = (Forty_df_lag['dm_lag_3']+Forty_df_lag['dm_lag_4']+Forty_df_lag['dm_lag_5'])/3
Forty_df_lag['SMA_7'] = (Forty_df_lag['dm_lag_4']+Forty_df_lag['dm_lag_5']+Forty_df_lag['dm_lag_6'])/3
Forty_df_lag['SMA_8'] = (Forty_df_lag['dm_lag_5']+Forty_df_lag['dm_lag_6']+Forty_df_lag['dm_lag_7'])/3

#drop demand lags, average monthly consumption, initial stock columns
Forty_df_lag = Forty_df_lag.drop(columns = ['average_monthly_consumption','stock_initial','product_type','site_type','region'],axis=1)

# Split NonZero_df_lag into Train, test and forecast
Forty_df_Train = Forty_df_lag[Forty_df_lag['tag']=='Train']
Forty_df_Test = Forty_df_lag[Forty_df_lag['tag']=='Test']
Forty_Forecast = Forty_df_lag[Forty_df_lag['tag']=='Forecast']

#Remove tag column
Forty_df_Train = Forty_df_Train.drop(columns = ['tag'],axis = 1)
Forty_df_Test = Forty_df_Test.drop(columns = ['tag'],axis = 1)
Forty_Forecast = Forty_Forecast.drop(columns = ['tag'],axis = 1)

Forty_df_Train = Forty_df_Train.fillna(0)
Forty_df_Test = Forty_df_Test.fillna(0)
Forty_Forecast = Forty_Forecast.fillna(0)

In [35]:
# Convert categorical features from object to category to fit light gbm model
NonZero_X_Forecast = NonZero_Forecast
NonZero_X_Forecast = NonZero_X_Forecast.set_index('date')
NonZero_X_Forecast = NonZero_X_Forecast.drop(columns=['value'],axis=1)

for c in NonZero_X_Forecast.columns:
    col_type = NonZero_X_Forecast[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        NonZero_X_Forecast[c] = NonZero_X_Forecast[c].astype('category')

Forty_Sixty_df_X_Forecast = Forty_Sixty_df_Forecast
Forty_Sixty_df_X_Forecast = Forty_Sixty_df_X_Forecast.set_index('date')
Forty_Sixty_df_X_Forecast = Forty_Sixty_df_X_Forecast.drop(columns=['value'],axis=1)

for c in Forty_Sixty_df_X_Forecast.columns:
    col_type = Forty_Sixty_df_X_Forecast[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Forty_Sixty_df_X_Forecast[c] = Forty_Sixty_df_X_Forecast[c].astype('category')

Forty_X_Forecast = Forty_Forecast
Forty_X_Forecast = Forty_X_Forecast.set_index('date')
Forty_X_Forecast = Forty_X_Forecast.drop(columns=['value'],axis=1)

for c in Forty_X_Forecast.columns:
    col_type = Forty_X_Forecast[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Forty_X_Forecast[c] = Forty_X_Forecast[c].astype('category')

Sixty_X_Forecast = Sixty_Forecast
Sixty_X_Forecast = Sixty_X_Forecast.set_index('date')
Sixty_X_Forecast = Sixty_X_Forecast.drop(columns=['value'],axis=1)

for c in Sixty_X_Forecast.columns:
    col_type = Sixty_X_Forecast[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Sixty_X_Forecast[c] = Sixty_X_Forecast[c].astype('category')
        

In [36]:
#### Split Train and Test into features and target variable ####

# Non Zero df Train and Test
NonZero_X_Train = NonZero_df_Train.drop(columns=['value','date'],axis = 1)
NonZero_y_Train = NonZero_df_Train[['batchID','value']]

NonZero_X_Test = NonZero_df_Test.drop(columns = ['value','date'],axis = 1)
NonZero_y_Test = NonZero_df_Test[['batchID','value']]

# Convert the categorical features from object type to category to fit lightgbm model
for c in NonZero_X_Train.columns:
    col_type = NonZero_X_Train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        NonZero_X_Train[c] = NonZero_X_Train[c].astype('category')
        
for c in NonZero_X_Test.columns:
    col_type = NonZero_X_Test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        NonZero_X_Test[c] = NonZero_X_Test[c].astype('category')

# Combine Train and test to fit on entire data
NonZero_full_X_train = pd.concat([NonZero_X_Train,NonZero_X_Test])
NonZero_full_y_train = pd.concat([NonZero_y_Train,NonZero_y_Test])

# Forty df Train and Test
Forty_X_Train = Forty_df_Train.drop(columns=['value','date'],axis = 1)
Forty_y_Train = Forty_df_Train[['batchID','value']]

Forty_X_Test = Forty_df_Test.drop(columns = ['value','date'],axis = 1)
Forty_y_Test = Forty_df_Test[['batchID','value']]

# Convert the categorical features from object type to category to fit lightgbm model
for c in Forty_X_Train.columns:
    col_type = Forty_X_Train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Forty_X_Train[c] = Forty_X_Train[c].astype('category')
        
for c in Forty_X_Test.columns:
    col_type = Forty_X_Test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Forty_X_Test[c] = Forty_X_Test[c].astype('category')

# Combine Train and test to fit on entire data
Forty_full_X_train = pd.concat([Forty_X_Train,Forty_X_Test])
Forty_full_y_train = pd.concat([Forty_y_Train,Forty_y_Test])
                
# Sixty df Train and Test
Sixty_X_Train = Sixty_df_Train.drop(columns=['value','date'],axis = 1)
Sixty_y_Train = Sixty_df_Train[['batchID','value']]

Sixty_X_Test = Sixty_df_Test.drop(columns = ['value','date'],axis = 1)
Sixty_y_Test = Sixty_df_Test[['batchID','value']]

# Convert the categorical features from object type to category to fit lightgbm model
for c in Sixty_X_Train.columns:
    col_type = Sixty_X_Train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Sixty_X_Train[c] = Sixty_X_Train[c].astype('category')
        
for c in Sixty_X_Test.columns:
    col_type = Sixty_X_Test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Sixty_X_Test[c] = Sixty_X_Test[c].astype('category')

# Combine Train and test to fit on entire data
Sixty_full_X_train = pd.concat([Sixty_X_Train,Sixty_X_Test])
Sixty_full_y_train = pd.concat([Sixty_y_Train,Sixty_y_Test])

# Forty df Train and Test
Forty_Sixty_df_X_Train = Forty_Sixty_df_Train.drop(columns=['value','date'],axis = 1)
Forty_Sixty_df_y_Train = Forty_Sixty_df_Train[['batchID','value']]

Forty_Sixty_df_X_Test = Forty_Sixty_df_Test.drop(columns = ['value','date'],axis = 1)
Forty_Sixty_df_y_Test = Forty_Sixty_df_Test[['batchID','value']]

# Convert the categorical features from object type to category to fit lightgbm model
for c in Forty_Sixty_df_X_Train.columns:
    col_type = Forty_Sixty_df_X_Train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Forty_Sixty_df_X_Train[c] = Forty_Sixty_df_X_Train[c].astype('category')
        
for c in Forty_Sixty_df_X_Test.columns:
    col_type = Forty_Sixty_df_X_Test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Forty_Sixty_df_X_Test[c] = Forty_Sixty_df_X_Test[c].astype('category')

# Combine Train and test to fit on entire data
Forty_Sixty_full_X_train = pd.concat([Forty_Sixty_df_X_Train,Forty_Sixty_df_X_Test])
Forty_Sixty_full_y_train = pd.concat([Forty_Sixty_df_y_Train,Forty_Sixty_df_y_Test])


In [37]:
#Fit a global catboost model - NonZero
NonZero_X_train_gbm = NonZero_X_Train.drop(columns = ['batchID'],axis = 1)
NonZero_y_train_gbm = NonZero_y_Train.drop(columns = ['batchID'],axis = 1)

from catboost import CatBoostRegressor, Pool

# Create a list of column names
cat_cols = NonZero_X_train_gbm.select_dtypes(include=['category'])
    
categorical_features = [NonZero_X_train_gbm.columns.tolist().index(x) for x in cat_cols]

train_pool = Pool(NonZero_X_train_gbm, label=NonZero_y_train_gbm,cat_features=categorical_features)

cb_tweedie = CatBoostRegressor(loss_function='Tweedie:variance_power=1.1', n_estimators=500, silent=True,random_seed=153)
cb_tweedie.fit(train_pool)#, eval_set=test_pool)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = NonZero_X_Test['batchID'].unique()

catboost_mae = []
batchID = []
result_dict = {}
NonZero_Catboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = NonZero_X_Test[NonZero_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = NonZero_y_Test[NonZero_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    test_pool = Pool(df_X_test,cat_features=categorical_features) #label=df_y_test
    
    #predict on test data
    cb_tweedie_pred = cb_tweedie.predict(test_pool)
        
    test_results['prediction'] = cb_tweedie_pred
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    NonZero_Catboost_Forecasts = NonZero_Catboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, cb_tweedie_pred)
    
    #Append MAE and batchID to list
    catboost_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':catboost_mae}

NonZero_catboost_MAE = pd.DataFrame(result_dict)    

In [38]:
#Fit a global lightgbm model
NonZero_X_train_gbm = NonZero_X_Train.drop(columns = ['batchID'],axis = 1)
NonZero_y_train_gbm = NonZero_y_Train.drop(columns = ['batchID'],axis = 1)

#Light gbm
import lightgbm as lgb

#Fit light gbm with tweedie loss function
model = lgb.LGBMRegressor(objective="tweedie",random_state=153,tweedie_variance_power = 1.1)
model.fit(NonZero_X_train_gbm, NonZero_y_train_gbm)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = NonZero_X_Test['batchID'].unique()

lightgbm_mae = []
batchID = []
result_dict = {}
NonZero_lightgbm_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = NonZero_X_Test[NonZero_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = NonZero_y_Test[NonZero_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    #predict on test data
    preds = model.predict(df_X_test)
        
    test_results['prediction'] = preds
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    NonZero_lightgbm_Forecasts = NonZero_lightgbm_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, preds)
    
    #Append MAE and batchID to list
    lightgbm_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':lightgbm_mae}

NonZero_lightgbm_MAE = pd.DataFrame(result_dict)    

In [39]:
#Fit a global catboost model - NonZero
Forty_Sixty_df_X_train_gbm = Forty_Sixty_df_X_Train.drop(columns = ['batchID'],axis = 1)
Forty_Sixty_df_y_train_gbm = Forty_Sixty_df_y_Train.drop(columns = ['batchID'],axis = 1)

from catboost import CatBoostRegressor, Pool

# Create a list of column names
cat_cols = Forty_Sixty_df_X_train_gbm.select_dtypes(include=['category'])
    
categorical_features = [Forty_Sixty_df_X_train_gbm.columns.tolist().index(x) for x in cat_cols]

train_pool = Pool(Forty_Sixty_df_X_train_gbm, label=Forty_Sixty_df_y_train_gbm,cat_features=categorical_features)

cb_tweedie = CatBoostRegressor(loss_function='Tweedie:variance_power=1.1', n_estimators=500, silent=True,random_seed=153)
cb_tweedie.fit(train_pool)#, eval_set=test_pool)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = Forty_Sixty_df_X_Test['batchID'].unique()

catboost_mae = []
batchID = []
result_dict = {}
Forty_Sixty_df_Catboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = Forty_Sixty_df_X_Test[Forty_Sixty_df_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = Forty_Sixty_df_y_Test[Forty_Sixty_df_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    test_pool = Pool(df_X_test,cat_features=categorical_features) #label=df_y_test
    
    #predict on test data
    cb_tweedie_pred = cb_tweedie.predict(test_pool)
        
    test_results['prediction'] = cb_tweedie_pred
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_Sixty_df_Catboost_Forecasts = Forty_Sixty_df_Catboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, cb_tweedie_pred)
    
    #Append MAE and batchID to list
    catboost_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':catboost_mae}

Forty_Sixty_df_catboost_MAE = pd.DataFrame(result_dict)    

In [40]:
#Fit a global lightgbm model
Forty_Sixty_df_X_train_gbm = Forty_Sixty_df_X_Train.drop(columns = ['batchID'],axis = 1)
Forty_Sixty_df_y_train_gbm = Forty_Sixty_df_y_Train.drop(columns = ['batchID'],axis = 1)

#Light gbm
import lightgbm as lgb

#Fit light gbm with tweedie loss function
model = lgb.LGBMRegressor(objective="tweedie",random_state=153,tweedie_variance_power = 1.1)
model.fit(Forty_Sixty_df_X_train_gbm, Forty_Sixty_df_y_train_gbm)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = Forty_Sixty_df_X_Test['batchID'].unique()

lightgbm_mae = []
batchID = []
result_dict = {}
Forty_Sixty_df_lightgbm_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = Forty_Sixty_df_X_Test[Forty_Sixty_df_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = Forty_Sixty_df_y_Test[Forty_Sixty_df_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    #predict on test data
    preds = model.predict(df_X_test)
        
    test_results['prediction'] = preds
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_Sixty_df_lightgbm_Forecasts = Forty_Sixty_df_lightgbm_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, preds)
    
    #Append MAE and batchID to list
    lightgbm_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':lightgbm_mae}

Forty_Sixty_df_lightgbm_MAE = pd.DataFrame(result_dict)    

In [41]:
#Fit a global catboost model - NonZero
Forty_X_train_gbm = Forty_X_Train.drop(columns = ['batchID'],axis = 1)
Forty_y_train_gbm = Forty_y_Train.drop(columns = ['batchID'],axis = 1)

from catboost import CatBoostRegressor, Pool

# Create a list of column names
cat_cols = Forty_X_train_gbm.select_dtypes(include=['category'])
    
categorical_features = [Forty_X_train_gbm.columns.tolist().index(x) for x in cat_cols]

train_pool = Pool(Forty_X_train_gbm, label=Forty_y_train_gbm,cat_features=categorical_features)

cb_tweedie = CatBoostRegressor(loss_function='Tweedie:variance_power=1.1', n_estimators=500, silent=True,random_seed=153)
cb_tweedie.fit(train_pool)#, eval_set=test_pool)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = Forty_X_Test['batchID'].unique()

catboost_mae = []
batchID = []
result_dict = {}
Forty_Catboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = Forty_X_Test[Forty_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = Forty_y_Test[Forty_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    test_pool = Pool(df_X_test,cat_features=categorical_features) #label=df_y_test
    
    #predict on test data
    cb_tweedie_pred = cb_tweedie.predict(test_pool)
        
    test_results['prediction'] = cb_tweedie_pred
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_Catboost_Forecasts = Forty_Catboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, cb_tweedie_pred)
    
    #Append MAE and batchID to list
    catboost_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':catboost_mae}

Forty_catboost_MAE = pd.DataFrame(result_dict)    

In [42]:
#Fit a global lightgbm model
Forty_X_train_gbm = Forty_X_Train.drop(columns = ['batchID'],axis = 1)
Forty_y_train_gbm = Forty_y_Train.drop(columns = ['batchID'],axis = 1)

#Light gbm
import lightgbm as lgb

#Fit light gbm with tweedie loss function
model = lgb.LGBMRegressor(objective="tweedie",random_state=153,tweedie_variance_power = 1.1)
model.fit(Forty_X_train_gbm, Forty_y_train_gbm)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = Forty_X_Test['batchID'].unique()

lightgbm_mae = []
batchID = []
result_dict = {}
Forty_lightgbm_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = Forty_X_Test[Forty_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = Forty_y_Test[Forty_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    #predict on test data
    preds = model.predict(df_X_test)
        
    test_results['prediction'] = preds
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_lightgbm_Forecasts = Forty_lightgbm_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, preds)
    
    #Append MAE and batchID to list
    lightgbm_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':lightgbm_mae}

Forty_lightgbm_MAE = pd.DataFrame(result_dict)    

In [43]:
#Fit a global catboost model - NonZero
Sixty_X_train_gbm = Sixty_X_Train.drop(columns = ['batchID'],axis = 1)
Sixty_y_train_gbm = Sixty_y_Train.drop(columns = ['batchID'],axis = 1)

from catboost import CatBoostRegressor, Pool

# Create a list of column names
cat_cols = Sixty_X_train_gbm.select_dtypes(include=['category'])
    
categorical_features = [Sixty_X_train_gbm.columns.tolist().index(x) for x in cat_cols]

train_pool = Pool(Sixty_X_train_gbm, label=Sixty_y_train_gbm,cat_features=categorical_features)

cb_tweedie = CatBoostRegressor(loss_function='Tweedie:variance_power=1.1', n_estimators=500, silent=True,random_seed=153)
cb_tweedie.fit(train_pool)#, eval_set=test_pool)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = Sixty_X_Test['batchID'].unique()

catboost_mae = []
batchID = []
result_dict = {}
Sixty_Catboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = Sixty_X_Test[Sixty_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = Sixty_y_Test[Sixty_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    test_pool = Pool(df_X_test,cat_features=categorical_features) #label=df_y_test
    
    #predict on test data
    cb_tweedie_pred = cb_tweedie.predict(test_pool)
        
    test_results['prediction'] = cb_tweedie_pred
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Sixty_Catboost_Forecasts = Sixty_Catboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, cb_tweedie_pred)
    
    #Append MAE and batchID to list
    catboost_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':catboost_mae}

Sixty_catboost_MAE = pd.DataFrame(result_dict)    

In [44]:
#Fit a global lightgbm model
Sixty_X_train_gbm = Sixty_X_Train.drop(columns = ['batchID'],axis = 1)
Sixty_y_train_gbm = Sixty_y_Train.drop(columns = ['batchID'],axis = 1)

#Light gbm
import lightgbm as lgb

#Fit light gbm with tweedie loss function
model = lgb.LGBMRegressor(objective="tweedie",random_state=153,tweedie_variance_power = 1.1)
model.fit(Sixty_X_train_gbm, Sixty_y_train_gbm)

#Predict on test data and calculate out of sample Mean absolute error for each batchID group
# select unique batch_id
batch_ids = Sixty_X_Test['batchID'].unique()

lightgbm_mae = []
batchID = []
result_dict = {}
Sixty_lightgbm_Forecasts = pd.DataFrame()

for i in batch_ids:
    df_X_test = Sixty_X_Test[Sixty_X_Test['batchID'] == i]
    df_X_test = df_X_test.drop(columns= ['batchID'],axis=1)
    
    df_y_test = test_results = Sixty_y_Test[Sixty_y_Test['batchID'] == i]
    df_y_test = df_y_test.drop(columns= ['batchID'],axis=1)
    
    #predict on test data
    preds = model.predict(df_X_test)
        
    test_results['prediction'] = preds
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Sixty_lightgbm_Forecasts = Sixty_lightgbm_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(df_y_test, preds)
    
    #Append MAE and batchID to list
    lightgbm_mae.append(MAE)
    batchID.append(i)

result_dict = {'batchID':batchID,
               'MAE':lightgbm_mae}

Sixty_lightgbm_MAE = pd.DataFrame(result_dict)    

In [45]:
# One hot encoding 
cols = ['district','site_code','product_code','year','month','quarter']

# NonZero 
NonZero_X_Train['tag'] = 'Train'
NonZero_X_Test['tag'] = 'Test'
NonZero_X_Forecast['tag'] = 'Forecast'

NonZero_df_rev = pd.concat([NonZero_X_Train,NonZero_X_Test,NonZero_X_Forecast])

# One-hot encode the data using pandas get_dummies
NonZero_df_rev = pd.get_dummies(NonZero_df_rev,columns=cols,drop_first=True)

NonZero_X_Train = NonZero_df_rev[NonZero_df_rev['tag'] == 'Train']
NonZero_X_Test = NonZero_df_rev[NonZero_df_rev['tag'] == 'Test']
NonZero_X_Forecast = NonZero_df_rev[NonZero_df_rev['tag'] == 'Forecast']

## Forty Sixty df
Forty_Sixty_df_X_Train['tag'] = 'Train'
Forty_Sixty_df_X_Test['tag'] = 'Test'
Forty_Sixty_df_X_Forecast['tag'] = 'Forecast'

Forty_Sixty_df_rev = pd.concat([Forty_Sixty_df_X_Train,Forty_Sixty_df_X_Test,Forty_Sixty_df_X_Forecast])

# One-hot encode the data using pandas get_dummies
Forty_Sixty_df_rev = pd.get_dummies(Forty_Sixty_df_rev,columns=cols,drop_first=True)

Forty_Sixty_df_X_Train = Forty_Sixty_df_rev[Forty_Sixty_df_rev['tag'] == 'Train']
Forty_Sixty_df_X_Test = Forty_Sixty_df_rev[Forty_Sixty_df_rev['tag'] == 'Test']
Forty_Sixty_df_X_Forecast = Forty_Sixty_df_rev[Forty_Sixty_df_rev['tag'] == 'Forecast']

## Sixty df
Sixty_X_Train['tag'] = 'Train'
Sixty_X_Test['tag'] = 'Test'
Sixty_X_Forecast['tag'] = 'Forecast'

Sixty_df_rev = pd.concat([Sixty_X_Train,Sixty_X_Test,Sixty_X_Forecast])

# One-hot encode the data using pandas get_dummies
Sixty_df_rev = pd.get_dummies(Sixty_df_rev,columns=cols,drop_first=True)

Sixty_X_Train = Sixty_df_rev[Sixty_df_rev['tag'] == 'Train']
Sixty_X_Test = Sixty_df_rev[Sixty_df_rev['tag'] == 'Test']
Sixty_X_Forecast = Sixty_df_rev[Sixty_df_rev['tag'] == 'Forecast']

# Forty df
Forty_X_Train['tag'] = 'Train'
Forty_X_Test['tag'] = 'Test'
Forty_X_Forecast['tag'] = 'Forecast'

Forty_df_rev = pd.concat([Forty_X_Train,Forty_X_Test,Forty_X_Forecast])

# One-hot encode the data using pandas get_dummies
Forty_df_rev = pd.get_dummies(Forty_df_rev,columns=cols,drop_first=True)

Forty_X_Train = Forty_df_rev[Forty_df_rev['tag'] == 'Train']
Forty_X_Test = Forty_df_rev[Forty_df_rev['tag'] == 'Test']
Forty_X_Forecast = Forty_df_rev[Forty_df_rev['tag'] == 'Forecast']

In [46]:
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict,TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [47]:
best_params = {'max_depth': 6, 'n_estimators': 100}

# Grid Search with cross validation
tscv = TimeSeriesSplit(n_splits=5)

#Fit Random Forest Regressor with grid search best params
rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],random_state=153, verbose=False)

#### Non Zero df ####

# Split train and test to Xtrain, Xtest, ytrain, ytest
NonZero_X_Train = NonZero_X_Train.drop(columns=['batchID','tag'],axis = 1)
NonZero_y_Train = NonZero_y_Train.drop(columns = ['batchID'],axis=1)

# Cross Validation with Time Series validation
NonZero_scores = cross_val_score(rfr, NonZero_X_Train,NonZero_y_Train, cv=tscv, scoring='neg_mean_absolute_error')

### Forty Sixty df #####
# Split train and test to Xtrain, Xtest, ytrain, ytest
Forty_Sixty_df_X_Train = Forty_Sixty_df_X_Train.drop(columns=['batchID','tag'],axis = 1)
Forty_Sixty_df_y_Train = Forty_Sixty_df_y_Train.drop(columns = ['batchID'],axis=1)

# Cross Validation with Time Series validation
Forty_Sixty_df_scores = cross_val_score(rfr, Forty_Sixty_df_X_Train,Forty_Sixty_df_y_Train, cv=tscv, scoring='neg_mean_absolute_error')

### Forty df #####

# Split train and test to Xtrain, Xtest, ytrain, ytest
Forty_X_Train = Forty_X_Train.drop(columns=['batchID','tag'],axis = 1)
Forty_y_Train = Forty_y_Train.drop(columns = ['batchID'],axis=1)

# Cross Validation with Time Series validation
Forty_scores = cross_val_score(rfr, Forty_X_Train,Forty_y_Train, cv=tscv, scoring='neg_mean_absolute_error')

##### Sixty df #######

# Split train and test to Xtrain, Xtest, ytrain, ytest
Sixty_X_Train = Sixty_X_Train.drop(columns=['batchID','tag'],axis = 1)
Sixty_y_Train = Sixty_y_Train.drop(columns = ['batchID'],axis=1)

# Cross Validation with Time Series validation
Sixty_scores = cross_val_score(rfr, Sixty_X_Train,Sixty_y_Train, cv=tscv, scoring='neg_mean_absolute_error')

In [48]:
batch_ids = np.unique(NonZero_X_Test['batchID'])

rf_mae = []
batchID = []
rf_result_dict = {}
NonZero_rf_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = NonZero_X_Test[NonZero_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = NonZero_y_Test[NonZero_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(rfr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    NonZero_rf_Forecasts = NonZero_rf_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    rf_mae.append(MAE)
    batchID.append(i)

rf_result_dict = {'batchID':batchID,
               'MAE':rf_mae}

NonZero_randomForest = pd.DataFrame(rf_result_dict)

In [49]:
batch_ids = np.unique(Forty_Sixty_df_X_Test['batchID'])

rf_mae = []
batchID = []
rf_result_dict = {}
Forty_Sixty_df_rf_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = Forty_Sixty_df_X_Test[Forty_Sixty_df_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = Forty_Sixty_df_y_Test[Forty_Sixty_df_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(rfr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_Sixty_df_rf_Forecasts = Forty_Sixty_df_rf_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    rf_mae.append(MAE)
    batchID.append(i)

rf_result_dict = {'batchID':batchID,
               'MAE':rf_mae}

Forty_Sixty_df_randomForest = pd.DataFrame(rf_result_dict)

In [50]:
batch_ids = np.unique(Forty_X_Test['batchID'])

rf_mae = []
batchID = []
rf_result_dict = {}
Forty_rf_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = Forty_X_Test[Forty_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = Forty_y_Test[Forty_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(rfr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_rf_Forecasts = Forty_rf_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    rf_mae.append(MAE)
    batchID.append(i)

rf_result_dict = {'batchID':batchID,
               'MAE':rf_mae}

Forty_randomForest = pd.DataFrame(rf_result_dict)

In [51]:
batch_ids = np.unique(Sixty_X_Test['batchID'])

rf_mae = []
batchID = []
rf_result_dict = {}
Sixty_rf_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = Sixty_X_Test[Sixty_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = Sixty_y_Test[Sixty_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(rfr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Sixty_rf_Forecasts = Sixty_rf_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    rf_mae.append(MAE)
    batchID.append(i)

rf_result_dict = {'batchID':batchID,
               'MAE':rf_mae}

Sixty_randomForest = pd.DataFrame(rf_result_dict)

In [52]:
import xgboost as xgb
xgbr = xgb.XGBRegressor(random_state=153,max_depth=6,n_estimators=25)

# Cross Validation with K fold validation
NonZero_xgbr_scores = cross_val_score(xgbr, NonZero_X_Train,NonZero_y_Train, cv=tscv, scoring='neg_mean_absolute_error')
Forty_Sixty_df_xgbr_scores = cross_val_score(xgbr, Forty_Sixty_df_X_Train,Forty_Sixty_df_y_Train, cv=tscv, scoring='neg_mean_absolute_error')
Forty_xgbr_scores = cross_val_score(xgbr, Forty_X_Train,Forty_y_Train, cv=tscv, scoring='neg_mean_absolute_error')
Sixty_xgbr_scores = cross_val_score(xgbr, Sixty_X_Train,Sixty_y_Train, cv=tscv, scoring='neg_mean_absolute_error')

In [53]:
batch_ids = np.unique(Forty_X_Test['batchID'])

xgb_mae = []
batchID = []
xgboost_result_dict = {}
Forty_xgboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = Forty_X_Test[Forty_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = Forty_y_Test[Forty_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(xgbr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_xgboost_Forecasts = Forty_xgboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    xgb_mae.append(MAE)
    batchID.append(i)

xgboost_result_dict = {'batchID':batchID,
               'MAE':xgb_mae}

Forty_xgboost = pd.DataFrame(xgboost_result_dict)

In [54]:
batch_ids = np.unique(NonZero_X_Test['batchID'])

xgb_mae = []
batchID = []
xgboost_result_dict = {}
NonZero_xgboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = NonZero_X_Test[NonZero_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = NonZero_y_Test[NonZero_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(xgbr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    NonZero_xgboost_Forecasts = NonZero_xgboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    xgb_mae.append(MAE)
    batchID.append(i)

xgboost_result_dict = {'batchID':batchID,
               'MAE':xgb_mae}

NonZero_xgboost = pd.DataFrame(xgboost_result_dict)

In [55]:
batch_ids = np.unique(Forty_Sixty_df_X_Test['batchID'])

xgb_mae = []
batchID = []
xgboost_result_dict = {}
Forty_Sixty_df_xgboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = Forty_Sixty_df_X_Test[Forty_Sixty_df_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = Forty_Sixty_df_y_Test[Forty_Sixty_df_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(xgbr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Forty_Sixty_df_xgboost_Forecasts = Forty_Sixty_df_xgboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    xgb_mae.append(MAE)
    batchID.append(i)

xgboost_result_dict = {'batchID':batchID,
               'MAE':xgb_mae}

Forty_Sixty_df_xgboost = pd.DataFrame(xgboost_result_dict)

In [56]:
batch_ids = np.unique(Sixty_X_Test['batchID'])

xgb_mae = []
batchID = []
xgboost_result_dict = {}
Sixty_xgboost_Forecasts = pd.DataFrame()

for i in batch_ids:
    X_test_rf = Sixty_X_Test[Sixty_X_Test['batchID'] == i]
    X_test_rf = X_test_rf.drop(columns = ['batchID','tag'],axis=1)
    y_test_rf = test_results = Sixty_y_Test[Sixty_y_Test['batchID'] == i]
    y_test_rf = y_test_rf.drop(columns=['batchID'],axis=1)
    
    predictions = cross_val_predict(xgbr, X_test_rf, y_test_rf, cv=3)
    
    test_results['prediction'] = predictions
    test_results['residual'] = test_results['value'] - test_results['prediction']
    test_results = test_results.rename(columns = {'value':'actual'})
    Sixty_xgboost_Forecasts = Sixty_xgboost_Forecasts.append(test_results)
    
    #Error Metrics - Mean Absolute Error
    from sklearn.metrics import mean_absolute_error

    MAE = mean_absolute_error(y_test_rf, predictions)
    
    #Append MAE and batchID to list
    xgb_mae.append(MAE)
    batchID.append(i)

xgboost_result_dict = {'batchID':batchID,
               'MAE':xgb_mae}

Sixty_xgboost = pd.DataFrame(xgboost_result_dict)

In [57]:
# Non Zero df - MAE Comparisons
NonZero_xgboost.columns = ['batchID','xgb_MAE']
NonZero_randomForest.columns = ['batchID','rf_MAE']
NonZero_lightgbm_MAE.columns = ['batchID','lgbm_MAE']
NonZero_catboost_MAE.columns = ['batchID','cb_MAE']
NonZero_Croston_MAE = NonZero_Croston_MAE.reset_index()

# Non Zero results
NonZero_results = pd.merge(NonZero_df_Naive_results,NonZero_xgboost,how='inner',on='batchID')
NonZero_results = pd.merge(NonZero_results,NonZero_Croston_MAE,how='inner',on='batchID')
NonZero_results = pd.merge(NonZero_results,NonZero_catboost_MAE,how='inner',on='batchID')
NonZero_results = pd.merge(NonZero_results,NonZero_randomForest,how='inner',on='batchID')
NonZero_results = pd.merge(NonZero_results,NonZero_lightgbm_MAE,how='inner',on='batchID')
NonZero_results = pd.merge(NonZero_results,insample_naive,how='inner',on='batchID')

# Find minimum MAE for each batch ID 
NonZero_results['Min'] = NonZero_results[['rf_MAE','xgb_MAE','lgbm_MAE','cb_MAE','Naive_MAE','Croston_MAE']].min(axis=1)

# Find which model has minimum MAE
NonZero_results['MinMAE_Model'] = NonZero_results[['rf_MAE','xgb_MAE','lgbm_MAE','cb_MAE','Naive_MAE','Croston_MAE']].idxmin(axis=1)
NonZero_results['MASE'] = NonZero_results['Min']/NonZero_results['Insample-MAE']
print(NonZero_results['MASE'].mean())

2.4180976939685976


In [58]:
# Forty df - MAE Comparisons
Forty_Sixty_df_xgboost.columns = ['batchID','Forty_Sixty_df_xgb_MAE']
Forty_Sixty_df_randomForest.columns = ['batchID','Forty_Sixty_df_rf_MAE']
Forty_Sixty_df_lightgbm_MAE.columns = ['batchID','Forty_Sixty_df_lgbm_MAE']
Forty_Sixty_df_catboost_MAE.columns = ['batchID','Forty_Sixty_df_cb_MAE']
Forty_Sixty_df_Croston_MAE = Forty_Sixty_df_Croston_MAE.reset_index()
Forty_Sixty_df_Croston_MAE.columns = ['batchID','Forty_Sixty_df_Croston_MAE']
Forty_Sixty_df_Naive_results.columns = ['batchID','Forty_Sixty_df_Naive_MAE']

Sixty_xgboost.columns = ['batchID','Sixty_xgb_MAE']
Sixty_randomForest.columns = ['batchID','Sixty_rf_MAE']
Sixty_lightgbm_MAE.columns = ['batchID','Sixty_lgbm_MAE']
Sixty_catboost_MAE.columns = ['batchID','Sixty_cb_MAE']
Sixty_Croston_MAE = Sixty_Croston_MAE.reset_index()
Sixty_Croston_MAE.columns = ['batchID','Sixty_Croston_MAE']
Sixty_df_Naive_results.columns = ['batchID','Sixty_df_Naive_MAE']

Forty_xgboost.columns = ['batchID','Forty_xgb_MAE']
Forty_randomForest.columns = ['batchID','Forty_rf_MAE']
Forty_lightgbm_MAE.columns = ['batchID','Forty_lgbm_MAE']
Forty_catboost_MAE.columns = ['batchID','Forty_cb_MAE']
Forty_Croston_MAE = Forty_Croston_MAE.reset_index()
Forty_Croston_MAE.columns = ['batchID','Forty_Croston_MAE']
Forty_df_Naive_results.columns = ['batchID','Forty_df_Naive_MAE']

# Forty results
results = pd.merge(Forty_Sixty_df_Naive_results,Forty_Sixty_df_xgboost,how='inner',on='batchID')
results = pd.merge(results,Forty_Sixty_df_Croston_MAE,how='inner',on='batchID')
results = pd.merge(results,Forty_Sixty_df_catboost_MAE,how='inner',on='batchID')
results = pd.merge(results,Forty_Sixty_df_randomForest,how='inner',on='batchID')
results = pd.merge(results,Forty_Sixty_df_lightgbm_MAE,how='inner',on='batchID')


results = pd.merge(results,Sixty_df_Naive_results,how='left',on='batchID')
results = pd.merge(results,Sixty_xgboost,how='left',on='batchID')
results = pd.merge(results,Sixty_Croston_MAE,how='left',on='batchID')
results = pd.merge(results,Sixty_catboost_MAE,how='left',on='batchID')
results = pd.merge(results,Sixty_randomForest,how='left',on='batchID')
results = pd.merge(results,Sixty_lightgbm_MAE,how='left',on='batchID')

results = pd.merge(results,Forty_df_Naive_results,how='left',on='batchID')
results = pd.merge(results,Forty_xgboost,how='left',on='batchID')
results = pd.merge(results,Forty_Croston_MAE,how='left',on='batchID')
results = pd.merge(results,Forty_catboost_MAE,how='left',on='batchID')
results = pd.merge(results,Forty_randomForest,how='left',on='batchID')
results = pd.merge(results,Forty_lightgbm_MAE,how='left',on='batchID')
results = pd.merge(results,insample_naive,how='left',on='batchID')

# Find minimum MAE for each batch ID 
results['Min'] = results[['Forty_Sixty_df_xgb_MAE','Forty_Sixty_df_rf_MAE','Forty_Sixty_df_lgbm_MAE',
      'Forty_Sixty_df_cb_MAE','Forty_Sixty_df_Croston_MAE','Sixty_xgb_MAE','Sixty_rf_MAE',
      'Sixty_lgbm_MAE','Sixty_cb_MAE','Sixty_Croston_MAE','Forty_xgb_MAE','Forty_rf_MAE',
      'Forty_lgbm_MAE','Forty_cb_MAE','Forty_Croston_MAE','Forty_df_Naive_MAE','Sixty_df_Naive_MAE','Forty_Sixty_df_Naive_MAE']].min(axis=1)

# Find which model has minimum MAE
results['MinMAE_Model'] = results[['Forty_Sixty_df_xgb_MAE','Forty_Sixty_df_rf_MAE','Forty_Sixty_df_lgbm_MAE',
      'Forty_Sixty_df_cb_MAE','Forty_Sixty_df_Croston_MAE','Sixty_xgb_MAE','Sixty_rf_MAE',
      'Sixty_lgbm_MAE','Sixty_cb_MAE','Sixty_Croston_MAE','Forty_xgb_MAE','Forty_rf_MAE',
      'Forty_lgbm_MAE','Forty_cb_MAE','Forty_Croston_MAE','Forty_df_Naive_MAE','Sixty_df_Naive_MAE','Forty_Sixty_df_Naive_MAE']].idxmin(axis=1)
results['MASE'] = results['Min']/results['Insample-MAE']
print(results['MASE'].mean())

1.004103741131517


In [59]:
# Find Best model
best_model = pd.concat([results[['batchID','MASE','MinMAE_Model']].sort_values(by='MASE',ascending=False),NonZero_results[['batchID','MASE','MinMAE_Model']].sort_values(by='MASE',ascending=False)])

In [60]:
# Use the best model to forecast 
best_model['MinMAE_Model'].value_counts()

Forty_Sixty_df_rf_MAE         87
Naive_MAE                     81
Croston_MAE                   75
rf_MAE                        74
lgbm_MAE                      74
xgb_MAE                       61
Forty_Sixty_df_Croston_MAE    54
cb_MAE                        52
Forty_Sixty_df_xgb_MAE        49
Forty_df_Naive_MAE            40
Forty_Sixty_df_lgbm_MAE       40
Sixty_df_Naive_MAE            25
Forty_lgbm_MAE                25
Forty_Sixty_df_cb_MAE         24
Sixty_lgbm_MAE                15
Sixty_cb_MAE                  13
Forty_cb_MAE                  11
Forty_rf_MAE                   3
Sixty_rf_MAE                   2
Name: MinMAE_Model, dtype: int64